In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

### Binance API Base URL + Parameters:
#### Format: OHLC; oldest to newest 

https://api.binance.us/api/v3/klines "?"

* symbol: Trading pair (e.g., BTCUSDT).
* interval: Time interval (e.g., 1m, 5m, 15m, 1h, 12h, 1d).
* startTime: Start time in milliseconds (UNIX timestamp × 1000).
* endTime: End time in milliseconds (UNIX timestamp × 1000).
* limit: Number of candles to fetch (max 500).

https://api.binance.us/api/v3/klines?symbol=BTCUSDT&interval=12h&startTime=1717449600000&endTime=1717536000000

ps. "Klines" refers to candlestick (OHLC) data
ps. the ? separates the base URL from the query 
ps. Use the timestamp of the day at 00:00 as the startTime and timestamp of the next day at 00:00 as the endTime to ensure the last day is complete.
ps. use https://www.epochconverter.com/ to covert time to UNIX
ps. each kline datapoint has: Open time (timestamp), Open price, High price, Low price, Close price, Volume, Close time (timestamp), Quote asset volume, Number of trades, Taker buy base asset volume, Taker buy quote asset volume, Ignore (placeholder)

In [4]:
def fetch_binance_data(symbol, interval, startTime, endTime):
    base_url = "https://api.binance.us/api/v3/klines" #the base url and the input parameters below
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": startTime,
        "endTime": endTime,
    }
    response = requests.get(base_url, params=params) #fetch the data via requests library
    data = response.json()
    return pd.DataFrame({ # create a dataframe with the human date and corresponding closing price
        "time": [datetime.fromtimestamp(int(entry[0]) / 1000) for entry in data],
        "close": [float(entry[4]) for entry in data]
    })

### Analyzing Data

In [6]:
def analyze_data(master_data, test_data):
    up_success = 0
    down_success = 0
    up_total = 0
    down_total = 0

    for i in range(0, len(master_data), 2): # make sure we don't go over the amount of data we have
        if i + 1 >= len(master_data):
            break

        t0_master_price = master_data.iloc[i]['close'] # calcuate the daily percent change
        t1_master_price = master_data.iloc[i + 1]['close']
        master_change = ((t1_master_price - t0_master_price) / t0_master_price) * 100

        if abs(master_change) > 1:  #filter the changes of over 1% 
            corresponding_day = i // 2
            if corresponding_day + 1 >= len(test_data):
                continue

            t0_test_price = test_data.iloc[corresponding_day]['close'] # compare primary token's t_0 to secondary's t_1
            t1_test_price = test_data.iloc[corresponding_day + 1]['close']

            if master_change > 1: # main logic
                up_total += 1
                if t1_test_price > t0_test_price:
                    up_success += 1
            elif master_change < -1:
                down_total += 1
                if t1_test_price < t0_test_price:
                    down_success += 1

    up_success_rate = (up_success / up_total * 100) if up_total > 0 else 0 # calculate success rates
    down_success_rate = (down_success / down_total * 100) if down_total > 0 else 0
    
    print("Primary moved +1%: " + str(up_total) + " times") # print the results of the testing
    print("Primary moved -1%: " + str(down_total) + " times")
    print("Success for up movements: " + str(up_success) + " times")
    print("Success for down movements: " + str(down_success) + " times")
    
    return up_success_rate, down_success_rate

### User Input...

In [8]:
master_data = fetch_binance_data("BTCUSDT", "12h", 1692811488000, 1713893088000) # watch out for the syntax
test_data = fetch_binance_data("ETHUSDT", "12h", 1692811488000, 1713893088000)

In [9]:
up_success_rate, down_success_rate = analyze_data(master_data, test_data)

Primary moved +1%: 54 times
Primary moved -1%: 47 times
Success for up movements: 25 times
Success for down movements: 25 times


In [10]:
print(f"Success rate for primary up and secondary up: {up_success_rate:.2f}%")
print(f"Success rate for primary down and secondary down: {down_success_rate:.2f}%")

Success rate for primary up and secondary up: 46.30%
Success rate for primary down and secondary down: 53.19%


### Human date to UNIX x1000 for the user input

In [12]:
human_start_date = "2024-12-10 20:00:00"
unix_start_date_ms = int(datetime.strptime(human_start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
print(f"The UNIX timestamp for {human_start_date} is {unix_start_date_ms} milliseconds.")

The UNIX timestamp for 2024-12-10 20:00:00 is 1733878800000 milliseconds.


In [13]:
master_data

,time,close
0,2023-08-23 20:00:00,26447.56
1,2023-08-24 08:00:00,26175.01
2,2023-08-24 20:00:00,26106.12
3,2023-08-25 08:00:00,26064.10
4,2023-08-25 20:00:00,26054.53
...,...,...
483,2024-04-21 08:00:00,64957.64
484,2024-04-21 20:00:00,65895.19
485,2024-04-22 08:00:00,66850.00
486,2024-04-22 20:00:00,66013.84


In [14]:
test_data

,time,close
0,2023-08-23 20:00:00,1668.00
1,2023-08-24 08:00:00,1660.91
2,2023-08-24 20:00:00,1653.90
3,2023-08-25 08:00:00,1654.35
4,2023-08-25 20:00:00,1650.00
...,...,...
483,2024-04-21 08:00:00,3144.32
484,2024-04-21 20:00:00,3192.87
485,2024-04-22 08:00:00,3198.90
486,2024-04-22 20:00:00,3179.25
